In [38]:
# Importando bibliotecas necessárias
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import time

In [39]:
# Carregando os dados dos arquivos csv fornecidos
movies = pd.read_csv('../../data/movies.csv')
ratings = pd.read_csv('../../data/ratings.csv')

In [40]:
# Pré-processamento dos dados
# Verificar a estrutura dos dados

# Medir o tempo de execução do pré-processamento
start_time = time.time()
print(movies.head())
print(movies.info())

print(ratings.head())
print(ratings.info())

# fazer as transformações necessárias
movies['movieId'] = movies['movieId'].astype(int)
# Dividir a coluna 'genres' em varias colunas
genres = movies['genres'].str.get_dummies(sep='|')
movies = pd.concat([movies, genres], axis=1)

# Remover colunas desnecessárias
ratings.drop('timestamp', axis=1, inplace=True)

# Tratar valores nulos
print(movies.isnull().sum())
print(ratings.isnull().sum())

# Se houver valores ausentes, você pode remover as linhas ou preencher os valores ausentes
# Por exemplo, para preencher com a média dos valores existentes:
ratings.fillna(ratings.mean(), inplace=True)


end_time = time.time()
execution_time = (end_time - start_time)
 
print(f"Tempo de execução: {execution_time:.2f} segundos.")

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB
None
   userId  movieId  rating   timestamp
0       1       

In [41]:
# Criar uma matriz de classificação de usuarios para filmes
user_movie_ratings = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

In [42]:
# Calcular a similaridade do cosseno entre os usuários
user_similarity = cosine_similarity(user_movie_ratings)


In [44]:
# Criar um dataframe com as similaridades e índices de usuário
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_ratings.index, columns=user_movie_ratings.index)


In [47]:
# Criar uma função para fazer recomendações para um usuário específico
# Medir o uso de memória 
import psutil

def recommend_movies(user_id, num_recommendations=10):
    # Obter os usuários mais similares ao usuário de referência
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:num_recommendations + 1].index

    # Obter os filmes classificados por usuários similares
    recommended_movies = user_movie_ratings.loc[similar_users].mean().sort_values(ascending=False).index

    return recommended_movies

if __name__ == '__main__' : 
    user_id = 1
    
    # Medir o uso de memória antes da execução da função recommend_movies
    process = psutil.Process()
    memory_before = process.memory_info().rss
    
    recommended_movies = recommend_movies(user_id)
    
    # Medir o uso de memória após a execução da função recommend_movies
    memory_after = process.memory_info().rss
    memory_usage = (memory_after - memory_before)

    print(f"Recomendações de filmes para o usuário {user_id}:")
    print(recommended_movies)
    print(f"Tempo de execução: {execution_time:.2f} segundos.")
    print(f"Uso de memória: {memory_usage} bytes.")

Recomendações de filmes para o usuário 1:
Index([   318,   2571,    296,    110,    260,    356,    480,   4993,    858,
         2858,
       ...
         4753,   4754,   4756,   4757,   4759,   4760,   4761,   4765,   4766,
       149532],
      dtype='int64', name='movieId', length=10325)
Tempo de execução: 0.38 segundos.
Uso de memória: 4096 bytes.


In [ ]:
# Criar uma função para fazer recomendações para um usuário específico
def recommend_movies(user_id, num_recommendations=10):
    # Obter os usuários mais similares ao usuário de referência
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:num_recommendations + 1].index

    # Obter os filmes classificados por usuários similares
    recommended_movies = user_movie_ratings.loc[similar_users].mean().sort_values(ascending=False).index

    return recommended_movies

In [ ]:
# Teste da função de recomendação para um usuário específico
user_id = 1
recommended_movies = recommend_movies(user_id)
print(f"Recomendações de filmes para o usuario {user_id}:")
print(recommended_movies)

Recomendações de filmes para o usuario 1:
Index([   318,   2571,    296,    110,    260,    356,    480,   4993,    858,
         2858,
       ...
         4753,   4754,   4756,   4757,   4759,   4760,   4761,   4765,   4766,
       149532],
      dtype='int64', name='movieId', length=10325)
